In [4]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.model_selection import train_test_split

# 读取处理后的特征值

In [5]:
dataset1 = pd.read_csv('./GenerateData1.csv')
dataset2 = pd.read_csv('./GenerateData2.csv')
dataset3 = pd.read_csv('./GenerateData3.csv') 

# 替换为0 的值

In [6]:
dataset1.label.replace(-1, 0, inplace=True)
dataset2.label.replace(-1, 0, inplace=True)

# 去重合并表格并删除不需要得表格

In [7]:

dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset12 = pd.concat([dataset1, dataset2], axis=0)
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['user_id', 'label', 'day_gap_before', 'coupon_id', 'day_gap_after'], axis=1)

In [8]:
dataset3.drop_duplicates(inplace=True)
dataset3_preds = dataset3[['user_id', 'coupon_id', 'date_received']]
dataset3_x = dataset3.drop(['user_id', 'coupon_id', 'date_received', 'day_gap_before', 'day_gap_after'], axis=1)
# 获取数据
dataTrain = xgb.DMatrix(dataset12_x, label=dataset12_y)
dataTest = xgb.DMatrix(dataset3_x)

In [9]:
def myauc(test):
    testgroup = test.groupby(['coupon_id'])
    aucs = []
    for i in testgroup:
        tmpdf = i[1]
        if len(tmpdf['label'].unique()) != 2:
            continue
        fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred'], pos_label=1)
        aucs.append(auc(fpr, tpr))
    return np.average(aucs)

# xgboot模型 score:0.7885

In [11]:
params = {'booster': 'gbtree',
          'objective': 'rank:pairwise',
          'eval_metric': 'auc',
          'gamma': 0.1,
          'min_child_weight': 1.1,
          'max_depth': 5,
          'lambda': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'eta': 0.01,
          'tree_method': 'exact',
          'seed': 0,
          'nthread': 12
          }

In [12]:

watchlist = [(dataTrain, 'train')]
model = xgb.train(params, dataTrain, num_boost_round=3500, evals=watchlist)

[0]	train-auc:0.84293
[1]	train-auc:0.84883
[2]	train-auc:0.85255
[3]	train-auc:0.85333
[4]	train-auc:0.85568
[5]	train-auc:0.85745
[6]	train-auc:0.85878
[7]	train-auc:0.85870
[8]	train-auc:0.85880
[9]	train-auc:0.85984
[10]	train-auc:0.85987
[11]	train-auc:0.86002
[12]	train-auc:0.86076
[13]	train-auc:0.86201
[14]	train-auc:0.86183
[15]	train-auc:0.86177
[16]	train-auc:0.86163
[17]	train-auc:0.86178
[18]	train-auc:0.86189
[19]	train-auc:0.86213
[20]	train-auc:0.86241
[21]	train-auc:0.86291
[22]	train-auc:0.86289
[23]	train-auc:0.86290
[24]	train-auc:0.86273
[25]	train-auc:0.86275
[26]	train-auc:0.86256
[27]	train-auc:0.86234
[28]	train-auc:0.86241
[29]	train-auc:0.86239
[30]	train-auc:0.86230
[31]	train-auc:0.86240
[32]	train-auc:0.86262
[33]	train-auc:0.86250
[34]	train-auc:0.86266
[35]	train-auc:0.86266
[36]	train-auc:0.86261
[37]	train-auc:0.86256
[38]	train-auc:0.86267
[39]	train-auc:0.86273
[40]	train-auc:0.86298
[41]	train-auc:0.86305
[42]	train-auc:0.86299
[43]	train-auc:0.8631

[346]	train-auc:0.87543
[347]	train-auc:0.87546
[348]	train-auc:0.87552
[349]	train-auc:0.87556
[350]	train-auc:0.87559
[351]	train-auc:0.87561
[352]	train-auc:0.87569
[353]	train-auc:0.87576
[354]	train-auc:0.87584
[355]	train-auc:0.87587
[356]	train-auc:0.87588
[357]	train-auc:0.87593
[358]	train-auc:0.87595
[359]	train-auc:0.87599
[360]	train-auc:0.87601
[361]	train-auc:0.87607
[362]	train-auc:0.87609
[363]	train-auc:0.87617
[364]	train-auc:0.87620
[365]	train-auc:0.87625
[366]	train-auc:0.87633
[367]	train-auc:0.87638
[368]	train-auc:0.87646
[369]	train-auc:0.87647
[370]	train-auc:0.87651
[371]	train-auc:0.87655
[372]	train-auc:0.87659
[373]	train-auc:0.87662
[374]	train-auc:0.87667
[375]	train-auc:0.87670
[376]	train-auc:0.87679
[377]	train-auc:0.87684
[378]	train-auc:0.87689
[379]	train-auc:0.87689
[380]	train-auc:0.87694
[381]	train-auc:0.87695
[382]	train-auc:0.87700
[383]	train-auc:0.87708
[384]	train-auc:0.87711
[385]	train-auc:0.87717
[386]	train-auc:0.87723
[387]	train-auc:

[688]	train-auc:0.88638
[689]	train-auc:0.88640
[690]	train-auc:0.88642
[691]	train-auc:0.88643
[692]	train-auc:0.88646
[693]	train-auc:0.88649
[694]	train-auc:0.88650
[695]	train-auc:0.88653
[696]	train-auc:0.88655
[697]	train-auc:0.88658
[698]	train-auc:0.88661
[699]	train-auc:0.88664
[700]	train-auc:0.88666
[701]	train-auc:0.88668
[702]	train-auc:0.88670
[703]	train-auc:0.88674
[704]	train-auc:0.88677
[705]	train-auc:0.88678
[706]	train-auc:0.88682
[707]	train-auc:0.88685
[708]	train-auc:0.88687
[709]	train-auc:0.88689
[710]	train-auc:0.88692
[711]	train-auc:0.88694
[712]	train-auc:0.88697
[713]	train-auc:0.88698
[714]	train-auc:0.88701
[715]	train-auc:0.88703
[716]	train-auc:0.88705
[717]	train-auc:0.88708
[718]	train-auc:0.88710
[719]	train-auc:0.88711
[720]	train-auc:0.88715
[721]	train-auc:0.88717
[722]	train-auc:0.88719
[723]	train-auc:0.88722
[724]	train-auc:0.88725
[725]	train-auc:0.88726
[726]	train-auc:0.88728
[727]	train-auc:0.88729
[728]	train-auc:0.88731
[729]	train-auc:

[1029]	train-auc:0.89214
[1030]	train-auc:0.89216
[1031]	train-auc:0.89218
[1032]	train-auc:0.89219
[1033]	train-auc:0.89220
[1034]	train-auc:0.89223
[1035]	train-auc:0.89223
[1036]	train-auc:0.89224
[1037]	train-auc:0.89225
[1038]	train-auc:0.89226
[1039]	train-auc:0.89227
[1040]	train-auc:0.89229
[1041]	train-auc:0.89230
[1042]	train-auc:0.89232
[1043]	train-auc:0.89233
[1044]	train-auc:0.89235
[1045]	train-auc:0.89237
[1046]	train-auc:0.89238
[1047]	train-auc:0.89239
[1048]	train-auc:0.89241
[1049]	train-auc:0.89243
[1050]	train-auc:0.89244
[1051]	train-auc:0.89244
[1052]	train-auc:0.89246
[1053]	train-auc:0.89247
[1054]	train-auc:0.89249
[1055]	train-auc:0.89251
[1056]	train-auc:0.89252
[1057]	train-auc:0.89254
[1058]	train-auc:0.89255
[1059]	train-auc:0.89256
[1060]	train-auc:0.89258
[1061]	train-auc:0.89260
[1062]	train-auc:0.89261
[1063]	train-auc:0.89262
[1064]	train-auc:0.89263
[1065]	train-auc:0.89264
[1066]	train-auc:0.89265
[1067]	train-auc:0.89267
[1068]	train-auc:0.89268


[1357]	train-auc:0.89589
[1358]	train-auc:0.89590
[1359]	train-auc:0.89592
[1360]	train-auc:0.89593
[1361]	train-auc:0.89594
[1362]	train-auc:0.89595
[1363]	train-auc:0.89595
[1364]	train-auc:0.89596
[1365]	train-auc:0.89597
[1366]	train-auc:0.89597
[1367]	train-auc:0.89598
[1368]	train-auc:0.89599
[1369]	train-auc:0.89600
[1370]	train-auc:0.89600
[1371]	train-auc:0.89601
[1372]	train-auc:0.89602
[1373]	train-auc:0.89603
[1374]	train-auc:0.89603
[1375]	train-auc:0.89604
[1376]	train-auc:0.89604
[1377]	train-auc:0.89606
[1378]	train-auc:0.89606
[1379]	train-auc:0.89607
[1380]	train-auc:0.89608
[1381]	train-auc:0.89609
[1382]	train-auc:0.89610
[1383]	train-auc:0.89610
[1384]	train-auc:0.89611
[1385]	train-auc:0.89612
[1386]	train-auc:0.89612
[1387]	train-auc:0.89614
[1388]	train-auc:0.89614
[1389]	train-auc:0.89615
[1390]	train-auc:0.89616
[1391]	train-auc:0.89617
[1392]	train-auc:0.89618
[1393]	train-auc:0.89619
[1394]	train-auc:0.89621
[1395]	train-auc:0.89622
[1396]	train-auc:0.89622


...

[2013]	train-auc:0.90089
[2014]	train-auc:0.90090
[2015]	train-auc:0.90090
[2016]	train-auc:0.90090
[2017]	train-auc:0.90091
[2018]	train-auc:0.90092
[2019]	train-auc:0.90093
[2020]	train-auc:0.90093
[2021]	train-auc:0.90094
[2022]	train-auc:0.90094
[2023]	train-auc:0.90095
[2024]	train-auc:0.90096
[2025]	train-auc:0.90097
[2026]	train-auc:0.90097
[2027]	train-auc:0.90098
[2028]	train-auc:0.90099
[2029]	train-auc:0.90099
[2030]	train-auc:0.90100
[2031]	train-auc:0.90101
[2032]	train-auc:0.90101
[2033]	train-auc:0.90102
[2034]	train-auc:0.90102
[2035]	train-auc:0.90102
[2036]	train-auc:0.90103
[2037]	train-auc:0.90104
[2038]	train-auc:0.90105
[2039]	train-auc:0.90105
[2040]	train-auc:0.90106
[2041]	train-auc:0.90106
[2042]	train-auc:0.90107
[2043]	train-auc:0.90107
[2044]	train-auc:0.90108
[2045]	train-auc:0.90109
[2046]	train-auc:0.90109
[2047]	train-auc:0.90110
[2048]	train-auc:0.90111
[2049]	train-auc:0.90111
[2050]	train-auc:0.90112
[2051]	train-auc:0.90113
[2052]	train-auc:0.90113


[2341]	train-auc:0.90273
[2342]	train-auc:0.90274
[2343]	train-auc:0.90274
[2344]	train-auc:0.90275
[2345]	train-auc:0.90275
[2346]	train-auc:0.90275
[2347]	train-auc:0.90276
[2348]	train-auc:0.90277
[2349]	train-auc:0.90277
[2350]	train-auc:0.90278
[2351]	train-auc:0.90278
[2352]	train-auc:0.90279
[2353]	train-auc:0.90279
[2354]	train-auc:0.90280
[2355]	train-auc:0.90280
[2356]	train-auc:0.90281
[2357]	train-auc:0.90281
[2358]	train-auc:0.90281
[2359]	train-auc:0.90282
[2360]	train-auc:0.90282
[2361]	train-auc:0.90283
[2362]	train-auc:0.90284
[2363]	train-auc:0.90284
[2364]	train-auc:0.90284
[2365]	train-auc:0.90285
[2366]	train-auc:0.90285
[2367]	train-auc:0.90286
[2368]	train-auc:0.90286
[2369]	train-auc:0.90287
[2370]	train-auc:0.90288
[2371]	train-auc:0.90288
[2372]	train-auc:0.90288
[2373]	train-auc:0.90289
[2374]	train-auc:0.90290
[2375]	train-auc:0.90290
[2376]	train-auc:0.90291
[2377]	train-auc:0.90292
[2378]	train-auc:0.90292
[2379]	train-auc:0.90293
[2380]	train-auc:0.90293


[2669]	train-auc:0.90433
[2670]	train-auc:0.90433
[2671]	train-auc:0.90434
[2672]	train-auc:0.90434
[2673]	train-auc:0.90435
[2674]	train-auc:0.90435
[2675]	train-auc:0.90436
[2676]	train-auc:0.90436
[2677]	train-auc:0.90436
[2678]	train-auc:0.90437
[2679]	train-auc:0.90437
[2680]	train-auc:0.90437
[2681]	train-auc:0.90438
[2682]	train-auc:0.90438
[2683]	train-auc:0.90438
[2684]	train-auc:0.90438
[2685]	train-auc:0.90439
[2686]	train-auc:0.90439
[2687]	train-auc:0.90440
[2688]	train-auc:0.90440
[2689]	train-auc:0.90441
[2690]	train-auc:0.90441
[2691]	train-auc:0.90441
[2692]	train-auc:0.90442
[2693]	train-auc:0.90443
[2694]	train-auc:0.90443
[2695]	train-auc:0.90443
[2696]	train-auc:0.90444
[2697]	train-auc:0.90444
[2698]	train-auc:0.90445
[2699]	train-auc:0.90445
[2700]	train-auc:0.90446
[2701]	train-auc:0.90446
[2702]	train-auc:0.90446
[2703]	train-auc:0.90447
[2704]	train-auc:0.90448
[2705]	train-auc:0.90448
[2706]	train-auc:0.90448
[2707]	train-auc:0.90448
[2708]	train-auc:0.90449


[2997]	train-auc:0.90572
[2998]	train-auc:0.90572
[2999]	train-auc:0.90572
[3000]	train-auc:0.90573
[3001]	train-auc:0.90573
[3002]	train-auc:0.90573
[3003]	train-auc:0.90574
[3004]	train-auc:0.90574
[3005]	train-auc:0.90575
[3006]	train-auc:0.90575
[3007]	train-auc:0.90575
[3008]	train-auc:0.90575
[3009]	train-auc:0.90576
[3010]	train-auc:0.90576
[3011]	train-auc:0.90577
[3012]	train-auc:0.90577
[3013]	train-auc:0.90577
[3014]	train-auc:0.90578
[3015]	train-auc:0.90578
[3016]	train-auc:0.90579
[3017]	train-auc:0.90579
[3018]	train-auc:0.90580
[3019]	train-auc:0.90580
[3020]	train-auc:0.90580
[3021]	train-auc:0.90580
[3022]	train-auc:0.90580
[3023]	train-auc:0.90581
[3024]	train-auc:0.90581
[3025]	train-auc:0.90582
[3026]	train-auc:0.90582
[3027]	train-auc:0.90582
[3028]	train-auc:0.90583
[3029]	train-auc:0.90583
[3030]	train-auc:0.90584
[3031]	train-auc:0.90584
[3032]	train-auc:0.90585
[3033]	train-auc:0.90585
[3034]	train-auc:0.90585
[3035]	train-auc:0.90585
[3036]	train-auc:0.90586


[3325]	train-auc:0.90697
[3326]	train-auc:0.90697
[3327]	train-auc:0.90697
[3328]	train-auc:0.90698
[3329]	train-auc:0.90698
[3330]	train-auc:0.90699
[3331]	train-auc:0.90699
[3332]	train-auc:0.90699
[3333]	train-auc:0.90700
[3334]	train-auc:0.90700
[3335]	train-auc:0.90701
[3336]	train-auc:0.90701
[3337]	train-auc:0.90702
[3338]	train-auc:0.90702
[3339]	train-auc:0.90702
[3340]	train-auc:0.90702
[3341]	train-auc:0.90702
[3342]	train-auc:0.90703
[3343]	train-auc:0.90703
[3344]	train-auc:0.90703
[3345]	train-auc:0.90704
[3346]	train-auc:0.90704
[3347]	train-auc:0.90704
[3348]	train-auc:0.90704
[3349]	train-auc:0.90705
[3350]	train-auc:0.90705
[3351]	train-auc:0.90706
[3352]	train-auc:0.90706
[3353]	train-auc:0.90706
[3354]	train-auc:0.90706
[3355]	train-auc:0.90706
[3356]	train-auc:0.90707
[3357]	train-auc:0.90707
[3358]	train-auc:0.90707
[3359]	train-auc:0.90708
[3360]	train-auc:0.90708
[3361]	train-auc:0.90708
[3362]	train-auc:0.90709
[3363]	train-auc:0.90709
[3364]	train-auc:0.90709


In [13]:
model.save_model('./xgbmodel')


In [12]:
model = xgb.Booster(params)

In [13]:
model.load_model('./xgbmodel')

In [17]:
dataset3_preds1 = dataset3_preds.copy()
dataset3_preds1['label'] = model.predict(dataTest)

In [19]:
dataset3_preds1.label.head()

0   -1.927854
1    0.834743
2   -2.466245
3   -1.992080
4   -0.544283
Name: label, dtype: float32

In [21]:
dataset3_preds1.label = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(
    dataset3_preds1.label.values.reshape(-1, 1))
dataset3_preds1.sort_values(by=['coupon_id', 'label'], inplace=True)
dataset3_preds1.to_csv("./xgb_preds.csv", index=None, header=None)
print(dataset3_preds1.describe())

            user_id      coupon_id  date_received          label
count  1.128030e+05  112803.000000   1.128030e+05  112803.000000
mean   3.684618e+06    9064.658006   2.016072e+07       0.374507
std    2.126358e+06    4147.283515   9.017693e+00       0.130249
min    2.090000e+02       3.000000   2.016070e+07       0.000000
25%    1.843824e+06    5035.000000   2.016071e+07       0.292860
50%    3.683073e+06    9983.000000   2.016072e+07       0.355278
75%    5.525176e+06   13602.000000   2.016072e+07       0.443395
max    7.361024e+06   14045.000000   2.016073e+07       1.000000


In [22]:
dataset3_preds1.label.head()

88774    0.201625
58111    0.210430
25100    0.218126
79286    0.224153
59129    0.241302
Name: label, dtype: float32

In [17]:
model = xgb.Booster()
model.load_model('./xgbmodel')

In [18]:
temp = dataset12[['coupon_id', 'label']].copy()
temp['pred'] = model.predict(xgb.DMatrix(dataset12_x))
temp.pred = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(temp['pred'].values.reshape(-1, 1))
print(myauc(temp))

0.7733047598560868
